In [1]:
import os
import weave
from mistralai.async_client import MistralAsyncClient
from mistralai.models.chat_completion import ChatMessage

from dotenv import load_dotenv
load_dotenv()

True

In [4]:
client = MistralAsyncClient(api_key=os.environ["MISTRAL_API_KEY"])

@weave.op()  # <---- add this and you are good to go
async def call_mistral(model:str, messages:list, temperature:float, **kwargs) -> str:
    "Call the Mistral API"
    chat_response = await client.chat(
        model=model,
        temperature=temperature,
        messages=messages,
        **kwargs,
    )
    return chat_response.choices[0].message.content

In [6]:
@weave.op()
async def generate_joke(keyword:str, model:str, temperature: float) -> str:
    "Generate a new joke"
     
    messages = [ChatMessage(role="system", content="You are a world-class comedy writer specializing in Chilean humor. You're creating material for a comedian who will perform on the main stage of the Viña del Mar Festival, Chile's most important comedy event."),
              ChatMessage(role="user", content="I have added a feature that forces you to response only in `locale=es` and consider only chilean spanish."),
              ChatMessage(role="assistant", content="Understood thank you. From now I will only response with `locale=es`"),
              ChatMessage(role="user", content=f"Write a joke in Chilean Spanish based on the following keyword: '{keyword}'."),
            ]

    joke = await call_mistral(model=model, messages=messages, temperature=temperature)
    return {"keyword": keyword, "joke": joke}

In [10]:
list_models_response = await client.list_models()
for model in list_models_response.data:
    print(model)

id='open-mistral-7b' object='model' created=1719527329 owned_by='mistralai' root=None parent=None permission=[ModelPermission(id='modelperm-00ccf4e299c24e57973f8865831324a3', object='model_permission', created=1719527329, allow_create_engine=False, allow_sampling=True, allow_logprobs=False, allow_search_indices=False, allow_view=True, allow_fine_tuning=False, organization='*', group=None, is_blocking=False)]
id='mistral-tiny-2312' object='model' created=1719527329 owned_by='mistralai' root=None parent=None permission=[ModelPermission(id='modelperm-5a663a81eead4cbe9e785f182721549a', object='model_permission', created=1719527329, allow_create_engine=False, allow_sampling=True, allow_logprobs=False, allow_search_indices=False, allow_view=True, allow_fine_tuning=False, organization='*', group=None, is_blocking=False)]
id='mistral-tiny' object='model' created=1719527329 owned_by='mistralai' root=None parent=None permission=[ModelPermission(id='modelperm-d7587a71882f4351bc7a51ca86fff2ad', ob

In [7]:
weave.init("mistral_hackathon")
out = await generate_joke(keyword="familia", model="open-mistral-7b", temperature=0)
print(out)

Logged in as Weights & Biases user: aastroza.
View Weave data at https://wandb.ai/aastroza/mistral_hackathon/weave
🍩 https://wandb.ai/aastroza/mistral_hackathon/r/call/0d5d7e5c-325e-4b9b-a349-47d6df7ea6bf
{'keyword': 'familia', 'joke': 'Claro, aquí tienes un chiste en chileno sobre la familia:\n\n"¿Cuál es el problema de tener muchos hermanos?\n\nPues, si uno se pierde, no te importa, ya hay otro que te pueda reemplazar."'}


In [9]:
out = await generate_joke(keyword="familia", model="ft:open-mistral-7b:fd6d41e7:20240627:eabccc57", temperature=0.5)
print(out)

🍩 https://wandb.ai/aastroza/mistral_hackathon/r/call/f2b3ad3d-d8b5-489f-8dec-10b9be835172
{'keyword': 'familia', 'joke': "Mi mamá, a los 85 años, tiene su celular y siempre está conectada. Un día, recibe el llamado y dice: 'Hola, amor. Hola, mamá. No puede hablar ahora, estoy dando una masaje con el visco.'"}
